In [1]:
from bokeh.io import show, output_file
from bokeh.plotting import figure, output_file, show
import numpy as np

In [2]:
from bokeh.models import ColumnDataSource, Plot,DatetimeTickFormatter,FixedTicker

In [3]:
Datapath_f1 = 'C:\\Users\\z0043H7B\\Tien\\Data\\dubuque\\wacker\\exploreSPM\\'

In [4]:
import sys
sys.path.insert(0, Datapath_f1)

In [5]:
import Tools10
import numpy as np
import time
import os
import pandas as pd
%load_ext autoreload
%autoreload 10

In [6]:
# test use a time series figure
Datapath = '/Users/z0043H7B/Tien/Data/dubuque/Dubuque_080819/Wacker/'

In [7]:
import os
os.chdir(Datapath)

In [8]:
spm = Tools10.fastInSPM_fldin(Datapath)

In [9]:
spm.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16263532 entries, 2018-10-17 10:46:59 to 2019-03-14 10:25:39.500000
Data columns (total 3 columns):
events    int64
device    int64
type      float64
dtypes: float64(1), int64(2)
memory usage: 496.3 MB


In [10]:
spm.head()

,events,device,type
time,,,
2018-10-17 10:46:59,174,64,NaN
2018-10-17 10:46:59,175,1,NaN
2018-10-17 10:46:59,177,1,NaN
2018-10-17 10:46:59,177,2,NaN
2018-10-17 10:46:59,177,3,NaN


In [9]:
newdf=pd.read_csv('Wacker_15minDetAct.csv',encoding='utf-8',parse_dates=True,index_col=0)

In [10]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14208 entries, 2018-10-17 10:45:00 to 2019-03-14 10:30:00
Data columns (total 16 columns):
det33          14208 non-null int64
det35          14208 non-null int64
det37          14208 non-null int64
det39          14208 non-null int64
det41          14208 non-null int64
det43          14208 non-null int64
det45          14208 non-null int64
det47          14208 non-null int64
totalct        14208 non-null int64
month          14208 non-null int64
day            14208 non-null int64
hour           14208 non-null int64
minute         14208 non-null int64
day-of-week    14208 non-null int64
year           14208 non-null int64
min_bin        14208 non-null int64
dtypes: int64(16)
memory usage: 1.8 MB


In [11]:
# load the 8 cluster labels
EightClus= pd.read_csv('ClusteringNk_n8_label.csv')

In [12]:
EightClus.iloc[:,0].value_counts()

1    2718
7    2555
4    2277
2    2026
3    1677
6    1360
0    1039
5     554
Name: colummn, dtype: int64

In [13]:
dump=EightClus.colummn.values

In [14]:
set(dump)

{0, 1, 2, 3, 4, 5, 6, 7}

In [15]:
len(dump)

14206

In [16]:
print(dump[0],dump[len(dump)-1])

4 4


In [17]:
#pad the first and last row for consistency
dump=np.append(4,dump)
dump=np.append(dump,4)

In [18]:
len(dump)

14208

In [19]:
newdf['clu_lbl']=dump

In [20]:
newdf.clu_lbl.value_counts()

1    2718
7    2555
4    2279
2    2026
3    1677
6    1360
0    1039
5     554
Name: clu_lbl, dtype: int64

In [21]:
sum(newdf.clu_lbl.value_counts())

14208

In [39]:
totalx =0
for i in list(set(dump)):
    print(len(prep_clust(newdf,i)[0]))
    totalx += len(prep_clust(newdf,i)[0])

1039
2718
2026
1677
2279
554
1360
2555


In [146]:
print(totalx)

14208


In [41]:
# load the 4 cluster labels
FourClus= pd.read_csv('ClusterNk_n4_label.csv')

In [42]:
print(len(FourClus))

12190


In [64]:
FourClus.head()

,year,month,day,dow,hh,minute,t_bin,zeros,det1,det2,det3,det4,det5,det6,det7,det8,total_cnt,totalBins,Clst
0,2018,10,17,2,11,0,44,0,98,32,31,29,70,25,34,20,339,3,1
1,2018,10,17,2,11,15,45,0,123,32,32,34,88,22,33,22,386,3,1
2,2018,10,17,2,11,30,46,0,131,32,37,37,64,21,31,25,378,3,1
3,2018,10,17,2,11,45,47,0,118,30,39,40,76,20,25,29,377,3,1
4,2018,10,17,2,12,0,48,0,103,28,37,34,66,23,30,25,346,3,1


In [156]:
FourClus.tail()

,year,month,day,dow,hh,minute,t_bin,zeros,det1,det2,det3,det4,det5,det6,det7,det8,total_cnt,totalBins,Clst
12185,2019,3,14,3,9,15,37,0,116,25,42,17,201,23,37,15,476,3,0
12186,2019,3,14,3,9,30,38,0,111,27,37,18,210,25,43,13,484,3,0
12187,2019,3,14,3,9,45,39,0,108,29,36,21,180,25,49,19,467,3,0
12188,2019,3,14,3,10,0,40,0,115,27,34,19,103,21,42,21,382,3,1
12189,2019,3,14,3,10,15,41,0,69,18,21,12,48,14,25,16,223,3,2


In [43]:
# create time stamp
dummy=pd.DataFrame({'year': list(FourClus.year.values),
                    'month': list(FourClus.month.values),
                     'day': list(FourClus.day.values),
                    'hour': list(FourClus.hh.values),
                    'minute': list(FourClus.minute.values)
                   })

In [44]:
dummy2=pd.to_datetime(dummy)

In [45]:
dummy.index=dummy2

In [46]:
dummy['clu_lbl']=FourClus.Clst.values

In [48]:
dummy.head()

,year,month,day,hour,minute,clu_lbl
2018-10-17 11:00:00,2018,10,17,11,0,1
2018-10-17 11:15:00,2018,10,17,11,15,1
2018-10-17 11:30:00,2018,10,17,11,30,1
2018-10-17 11:45:00,2018,10,17,11,45,1
2018-10-17 12:00:00,2018,10,17,12,0,1


In [155]:
dummy.tail()

,year,month,day,hour,minute,clu_lbl
2019-03-14 09:15:00,2019,3,14,9,15,0
2019-03-14 09:30:00,2019,3,14,9,30,0
2019-03-14 09:45:00,2019,3,14,9,45,0
2019-03-14 10:00:00,2019,3,14,10,0,1
2019-03-14 10:15:00,2019,3,14,10,15,2


In [49]:
dummy.clu_lbl.value_counts()

1    3880
0    3315
3    2621
2    2374
Name: clu_lbl, dtype: int64

In [50]:
print(sum(dummy.clu_lbl.value_counts()))

12190


In [38]:
def prep_clust(df,nclus):
    """ function used to extract the timestamps for different cluster labeled bins"""
    # create new subset of data
    sub = df[df.clu_lbl==nclus]
    st_d = pd.to_datetime(sub.index)
    ed_d = st_d+pd.to_timedelta('15min')
    
    return st_d,ed_d

In [127]:
for i in list(set(dump)):
    print(len(prep_clust(newdf,i)[0]))

1039
2718
2026
1677
2279
554
1360
2555


In [132]:
set(dump)

{0, 1, 2, 3, 4, 5, 6, 7}

In [53]:
# plot total activity per 15 mins on a full figure
    # setup figure
output_file('Devon_car_cnt_actwClust8.html')
    # setup size 
p = figure(title='Wacker detectors count',plot_width=1400, plot_height=500)
    
    # setup tickers
mjtick, mntick= Tools10.gen_dtticker(newdf, '1H', '30T')
p.vbar(x=newdf.index, width=0.5, bottom=0, top=newdf['totalct'], color="#1d49cf")

## create line segment info for clusters
std,etd=prep_clust(newdf,0)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e8810c", line_width=7)
std,etd=prep_clust(newdf,1)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#9c09eb", line_width=7)
std,etd=prep_clust(newdf,2)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#3309eb", line_width=7)
std,etd=prep_clust(newdf,3)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#13c70a", line_width=7)
std,etd=prep_clust(newdf,4)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#c598ed", line_width=7)
std,etd=prep_clust(newdf,5)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e63715", line_width=7)
std,etd=prep_clust(newdf,6)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#ed8a77", line_width=7)
std,etd=prep_clust(newdf,7)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#a8d472", line_width=7)

### plotting the second line for 4 clusters
std,etd=prep_clust(dummy,0)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#edd924", line_width=7)
std,etd=prep_clust(dummy,1)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e8812c", line_width=7)
std,etd=prep_clust(dummy,2)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e64a27", line_width=7)
std,etd=prep_clust(dummy,3)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#306cdb", line_width=7)

p.xgrid.ticker = FixedTicker(ticks=mntick) # set minor ticker as more granularity
p.xaxis.ticker = FixedTicker(ticks=mjtick) #set major ticker to axis label
#p.xaxis.ticker.desired_num_ticks = 100
p.xaxis.major_label_orientation = "vertical"  # rotate label orientation
p.xaxis.formatter=DatetimeTickFormatter(days=["%m/%d %H:%M"],months=["%m/%d %H:%M"],hours=["%m/%d %H:%M"])#,minutes=["%m/%d %H:%M"])
    # set the grid interval filling
p.xgrid.band_fill_color = "grey"
p.xgrid.band_fill_alpha = 0.05
show(p)

In [ ]:
# do the stats part


In [55]:
# seperate out the workdays
Wkd = newdf[newdf['day-of-week'].isin([x for x in range(0,5)])]  # Mon-Friday
Wked = newdf[newdf['day-of-week'].isin([x for x in range(5,7)])]  # Sat,Sun

In [56]:
# exclude holidays from data:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

dr = pd.date_range(start='2018-10-17', end='2019-03-14')
tmpdf = pd.DataFrame()
tmpdf['Date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())
tmpdf['Holiday'] = tmpdf['Date'].isin(holidays)
tmpdf.index=pd.to_datetime(tmpdf['Date'])
# select out the holidays
holi_inddate=tmpdf[tmpdf.Holiday==True].index

In [57]:
print(holi_inddate)

DatetimeIndex(['2018-11-12', '2018-11-22', '2018-12-25', '2019-01-01',
               '2019-01-21', '2019-02-18'],
              dtype='datetime64[ns]', name='Date', freq=None)


In [58]:
Wkd['doy']=Wkd.index.dayofyear

C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
print("number of day excluded due to holiday in weekdays:",int(sum(Wkd.doy.isin(holi_inddate.dayofyear)/96)))

number of day excluded due to holiday in weekdays: 6


In [60]:
Wkd=Wkd[~Wkd.doy.isin(holi_inddate.dayofyear)] #get rid of the holidays in weekdays

In [61]:
# create unique day code
yr = Wkd.year.values
mon = Wkd.month.values
dd = Wkd.day.values

In [62]:
inp_unid = [str(yr[i])+str(mon[i])+str(dd[i]) for i in range(0,len(yr))]
Wkd['unid']=inp_unid    # create an unique id for the day for easy filtering 

In [63]:
totdys = len(set(Wkd.unid.values))
print('total number of weekdays used:',totdys)

total number of weekdays used: 101


In [64]:
# define the min bin group for two portions the day
Morn = [x for x in range(0,48)]  #0AM-12PM
#Midd = [x for x in range(44,65)]  **** Should do Midday peak??
Aft = [x for x in range(49,96)]

In [76]:
trimWkd=Wkd.drop(Wkd.index[[0,1,2,3,4]])

In [79]:
# calculate AM PM average cnts per day
avgM = pd.DataFrame(columns=['date','AM average car cnts'])
avgA = pd.DataFrame(columns=['date','PM average car cnts'])

AMavgct=[]
PMavgct=[]
AMdatet=[]
PMdatet=[]
for bin in range(0,200): # taking out each pktimes from different days
    # start by sub-filtering half day ***### shift middle of day to 14:00 and end of day to 2:00
    #print(bin)
    Wksubdf = trimWkd.iloc[bin*48:(bin*48)+48][['totalct','min_bin']]
    # find out first starting hour
    stddt =Wksubdf.index[0]

    # get to know which portion of day this sub is
    if Wksubdf.min_bin[0]>47: por=1 #afternoon, coded morning[0], afternoon[1]
    else: por=0
        
    tmp = list(Wksubdf.totalct.values)
    indx = tmp.index(max(tmp))
    if por==0:                                # the morning
        AMavgct+=[sum(Wksubdf.totalct.values)/48]
        AMdatet+=[Wksubdf.index[0]]
    else:                                    # the afternoon
        PMavgct+=[sum(Wksubdf.totalct.values)/48]
        PMdatet+=[Wksubdf.index[0]]

In [83]:
avgM['AM average car cnts']=AMavgct
avgA['PM average car cnts']=PMavgct
avgM['date']=AMdatet
avgA['date']=PMdatet

In [84]:
avgM.describe()

,AM average car cnts
count,100.000000
mean,96.860208
std,4.855239
min,72.916667
25%,94.921875
50%,98.197917
75%,99.739583
max,106.937500


In [85]:
avgA.describe()

,PM average car cnts
count,100.000000
mean,116.082708
std,5.824418
min,103.520833
25%,112.552083
50%,117.125000
75%,119.723958
max,129.770833


In [90]:
recM = pd.DataFrame(columns=['Peak min_bin','start bin','end bin','Peak time','start time','end time'])
recA = pd.DataFrame(columns=['Peak min_bin','start bin','end bin','Peak time','start time','end time'])

Mornpk=[]
Aftpk=[]
Mornpkt=[]
Aftpkt=[]
Mornst=[]
Aftst=[]
Mornstt=[]
Aftstt=[]
Morned=[]
Afted=[]
Mornedt=[]
Aftedt=[]
for bin in range(0,200): # taking out each pktimes from different days
    # start by sub-filtering half day
    Wksubdf = trimWkd.iloc[bin*48:(bin*48)+48][['totalct','min_bin']]
    # find out first starting hour
    stddt =Wksubdf.index[0]
    if(stddt.minute != 0):std = pd.Timestamp(stddt.year,stddt.month,stddt.day,stddt.hour)
    else: std = stddt
    # get to know which portion of day this sub is
    if Wksubdf.min_bin[0]>47: por=1 #afternoon, coded morning[0], afternoon[1]
    else: por=0
    
    # start from subdf and get peak timestamps
    tmp = list(Wksubdf.totalct.values)
    indx = tmp.index(max(tmp))
    if por==0:                                # the morning
        Mornpkt += [Wksubdf.index[indx]]
        Mornpk += [Wksubdf.min_bin[indx]]
        cut60 = AMavgct[int(bin/2)]#max(tmp)*.4 # or use 40% of peak?
        #print('AM',max(tmp),cut60)
        # find front portion cut off
        idtf=tmp[0:indx]>cut60
        fid=Tools10.asclstT(idtf)
        Mornstt += [Wksubdf.index[fid]]
        Mornst += [Wksubdf.min_bin[fid]]
        #print('AM start:',Wksubdf.totalct[fid-1])
        # find back portion cut off
        idtf=tmp[indx:len(tmp)]>cut60
        fid=Tools10.desclstT(idtf)
        Mornedt += [Wksubdf.index[indx+fid]]
        Morned +=  [Wksubdf.min_bin[indx+fid]]
        #print('AM end:',Wksubdf.totalct[indx+fid])
        
    else:                                    # the afternoon
        Aftpkt+= [Wksubdf.index[indx]]
        Aftpk += [Wksubdf.min_bin[indx]]
        cut60 = PMavgct[int(bin/2)]*.8 #max(tmp)*.5
        #print('PM',max(tmp),cut60)
        # find front portion cut off
        idtf=tmp[0:indx]>cut60
        fid=Tools10.asclstT(idtf)
        Aftstt += [Wksubdf.index[fid]]
        Aftst += [Wksubdf.min_bin[fid]]
        #print('PM start:',Wksubdf.totalct[fid])
        # find back portion cut off
        idtf=tmp[indx:len(tmp)]>cut60
        fid=Tools10.desclstT(idtf)
        Aftedt += [Wksubdf.index[indx+fid]]
        Afted += [Wksubdf.min_bin[indx+fid]]
        #print('PM end:',Wksubdf.totalct[indx+fid])

In [91]:
# fill in the Morning records
recM['Peak min_bin']=Mornpk
recM['start bin']=Mornst
recM['end bin']=Morned
recM['Peak time']=Mornpkt
recM['start time']=Mornstt
recM['end time']=Mornedt

In [92]:
recA['Peak min_bin']=Aftpk
recA['start bin']=Aftst
recA['end bin']=Afted
recA['Peak time']=Aftpkt
recA['start time']=Aftstt
recA['end time']=Aftedt

In [99]:
recA.head()

,Peak min_bin,start bin,end bin,Peak time,start time,end time
0,78,70,91,2018-10-17 19:30:00,2018-10-17 17:30:00,2018-10-17 22:45:00
1,79,70,90,2018-10-18 19:45:00,2018-10-18 17:30:00,2018-10-18 22:30:00
2,83,70,94,2018-10-19 20:45:00,2018-10-19 17:30:00,2018-10-19 23:30:00
3,78,48,87,2018-10-22 19:30:00,2018-10-22 12:00:00,2018-10-22 21:45:00
4,79,68,88,2018-10-23 19:45:00,2018-10-23 17:00:00,2018-10-23 22:00:00


In [100]:
recM.head()

,Peak min_bin,start bin,end bin,Peak time,start time,end time
0,28,22,47,2018-10-18 07:00:00,2018-10-18 05:30:00,2018-10-18 11:45:00
1,33,22,45,2018-10-19 08:15:00,2018-10-19 05:30:00,2018-10-19 11:15:00
2,26,22,47,2018-10-22 06:30:00,2018-10-22 05:30:00,2018-10-22 11:45:00
3,31,21,44,2018-10-23 07:45:00,2018-10-23 05:15:00,2018-10-23 11:00:00
4,33,21,44,2018-10-24 08:15:00,2018-10-24 05:15:00,2018-10-24 11:00:00


In [ ]:
## replotting of the same figure

In [97]:
# plot total activity per 15 mins on a full figure
    # setup figure
output_file('Wacker_car_cnt_actwClust8.html')
    # setup size 
p = figure(title='Wacker detectors count',plot_width=1400, plot_height=500)
    
    # setup tickers
mjtick, mntick= Tools10.gen_dtticker(newdf, '1H', '30T')
p.vbar(x=newdf.index, width=0.5, bottom=0, top=newdf['totalct'], color="#1d49cf")

## create line segment info for clusters
std,etd=prep_clust(newdf,0)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e8810c", line_width=7)
std,etd=prep_clust(newdf,1)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#9c09eb", line_width=7)
std,etd=prep_clust(newdf,2)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#3309eb", line_width=7)
std,etd=prep_clust(newdf,3)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#13c70a", line_width=7)
std,etd=prep_clust(newdf,4)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#c598ed", line_width=7)
std,etd=prep_clust(newdf,5)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e63715", line_width=7)
std,etd=prep_clust(newdf,6)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#ed8a77", line_width=7)
std,etd=prep_clust(newdf,7)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#a8d472", line_width=7)

### plotting the second line for 4 clusters
std,etd=prep_clust(dummy,0)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#edd924", line_width=7)
std,etd=prep_clust(dummy,1)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e8812c", line_width=7)
std,etd=prep_clust(dummy,2)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e64a27", line_width=7)
std,etd=prep_clust(dummy,3)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#306cdb", line_width=7)

#### plotting the peaktimes start and end
# plot peak times
y1=[230 for x in range(len(recM))]
y0=[0 for x in range(len(recM))]
p.segment(x0=recM['Peak time'], y0=y0, x1=recM['Peak time'], y1=y1, line_color="#eb4034", line_width=3,alpha=.3)
y1=[230 for x in range(len(recA))]
y0=[0 for x in range(len(recA))]
p.segment(x0=recA['Peak time'], y0=y0, x1=recA['Peak time'], y1=y1, line_color="#5327d9", line_width=3,alpha=.3)

## plotting the start and end
p.segment(x0=recA['start time'], y0=y0, x1=recA['start time'], y1=y1, line_color="#0e9941", line_width=3,alpha=.3)
p.segment(x0=recM['start time'], y0=y0, x1=recM['start time'], y1=y1, line_color="#0e9941", line_width=3,alpha=.3)
p.segment(x0=recA['end time'], y0=y0, x1=recA['end time'], y1=y1, line_color="#0e9941", line_width=3,alpha=.3)
p.segment(x0=recM['end time'], y0=y0, x1=recM['end time'], y1=y1, line_color="#0e9941", line_width=3,alpha=.3)



p.xgrid.ticker = FixedTicker(ticks=mntick) # set minor ticker as more granularity
p.xaxis.ticker = FixedTicker(ticks=mjtick) #set major ticker to axis label
#p.xaxis.ticker.desired_num_ticks = 100
p.xaxis.major_label_orientation = "vertical"  # rotate label orientation
p.xaxis.formatter=DatetimeTickFormatter(days=["%m/%d %H:%M"],months=["%m/%d %H:%M"],hours=["%m/%d %H:%M"])#,minutes=["%m/%d %H:%M"])
    # set the grid interval filling
p.xgrid.band_fill_color = "grey"
p.xgrid.band_fill_alpha = 0.05
show(p)

In [116]:
recM['AMstart str']=recM['start time'].dt.strftime('%H:%M')
recM['AMend str']=recM['end time'].dt.strftime('%H:%M')

In [119]:
print('time bin : the % for start time')
print(recM['AMstart str'].value_counts()/len(recM)*100)
print('time bin : the % for end time')
print(recM['AMend str'].value_counts()/len(recM)*100)

time bin : the % for start time
05:15    53.0
05:30    30.0
05:00     9.0
05:45     3.0
04:45     2.0
06:00     2.0
06:15     1.0
Name: AMstart str, dtype: float64
time bin : the % for end time
11:45    59.0
11:30    12.0
11:15    11.0
11:00    10.0
10:30     3.0
10:45     3.0
10:15     1.0
09:30     1.0
Name: AMend str, dtype: float64


In [122]:
recA['PMstart str']=recA['start time'].dt.strftime('%H:%M')
recA['PMend str']=recA['end time'].dt.strftime('%H:%M')

In [123]:
print('time bin : the % for start time')
print(recA['PMstart str'].value_counts()/len(recA)*100)
print('time bin : the % for end time')
print(recA['PMend str'].value_counts()/len(recA)*100)

time bin : the % for start time
12:00    21.0
17:45    16.0
17:30    16.0
17:15    12.0
17:00     6.0
18:00     6.0
16:00     5.0
16:45     3.0
18:15     2.0
13:00     2.0
16:30     2.0
18:30     1.0
16:15     1.0
23:00     1.0
14:00     1.0
12:30     1.0
15:45     1.0
13:45     1.0
15:00     1.0
15:30     1.0
Name: PMstart str, dtype: float64
time bin : the % for end time
22:15    20.0
22:30    19.0
22:00    13.0
23:00     8.0
21:30     7.0
23:15     6.0
22:45     6.0
21:45     6.0
21:00     4.0
23:45     4.0
23:30     4.0
21:15     1.0
20:00     1.0
20:30     1.0
Name: PMend str, dtype: float64


In [143]:
import datetime
stdM=pd.to_datetime(recM['start time'].dt.date)+pd.to_timedelta('5:15:00')
etdM=pd.to_datetime(recM['end time'].dt.date)+pd.to_timedelta('11:45:00')

In [147]:
stdA=pd.to_datetime(recA['start time'].dt.date)+pd.to_timedelta('17:30:00')
etdA=pd.to_datetime(recA['end time'].dt.date)+pd.to_timedelta('22:15:00')

In [148]:
# plot total activity per 15 mins on a full figure
    # setup figure
output_file('Wacker_car_cnt_actwClust8_wstat.html')
    # setup size 
p = figure(title='Wacker detectors count',plot_width=1400, plot_height=500)
    
    # setup tickers
mjtick, mntick= Tools10.gen_dtticker(newdf, '1H', '30T')
p.vbar(x=newdf.index, width=0.5, bottom=0, top=newdf['totalct'], color="#1d49cf")

## create line segment info for 8 clusters
std,etd=prep_clust(newdf,0)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e8810c", line_width=7)
std,etd=prep_clust(newdf,1)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#9c09eb", line_width=7)
std,etd=prep_clust(newdf,2)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#3309eb", line_width=7)
std,etd=prep_clust(newdf,3)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#13c70a", line_width=7)
std,etd=prep_clust(newdf,4)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#c598ed", line_width=7)
std,etd=prep_clust(newdf,5)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e63715", line_width=7)
std,etd=prep_clust(newdf,6)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#ed8a77", line_width=7)
std,etd=prep_clust(newdf,7)
y=[200 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#a8d472", line_width=7)

### plotting the second line for 4 clusters
std,etd=prep_clust(dummy,0)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#edd924", line_width=7)
std,etd=prep_clust(dummy,1)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e8812c", line_width=7)
std,etd=prep_clust(dummy,2)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#e64a27", line_width=7)
std,etd=prep_clust(dummy,3)
y=[210 for x in range(len(std))]
p.segment(x0=std, y0=y, x1=etd, y1=y, line_color="#306cdb", line_width=7)

#### plotting the peaktimes start and end
# plot peak times
y1=[230 for x in range(len(recM))]
y0=[0 for x in range(len(recM))]
p.segment(x0=recM['Peak time'], y0=y0, x1=recM['Peak time'], y1=y1, line_color="#eb4034", line_width=3,alpha=.3)
y1=[230 for x in range(len(recA))]
y0=[0 for x in range(len(recA))]
p.segment(x0=recA['Peak time'], y0=y0, x1=recA['Peak time'], y1=y1, line_color="#5327d9", line_width=3,alpha=.3)

## plotting the start and end
# create daily start time and end time

p.segment(x0=stdM, y0=y0, x1=stdM, y1=y1, line_color="#0e9941", line_width=3,alpha=.3)
p.segment(x0=etdM, y0=y0, x1=etdM, y1=y1, line_color="#0e9941", line_width=3,alpha=.3)
p.segment(x0=stdA, y0=y0, x1=stdA, y1=y1, line_color="#0e9941", line_width=3,alpha=.3)
p.segment(x0=etdA, y0=y0, x1=etdA, y1=y1, line_color="#0e9941", line_width=3,alpha=.3)


p.xgrid.ticker = FixedTicker(ticks=mntick) # set minor ticker as more granularity
p.xaxis.ticker = FixedTicker(ticks=mjtick) #set major ticker to axis label
#p.xaxis.ticker.desired_num_ticks = 100
p.xaxis.major_label_orientation = "vertical"  # rotate label orientation
p.xaxis.formatter=DatetimeTickFormatter(days=["%m/%d %H:%M"],months=["%m/%d %H:%M"],hours=["%m/%d %H:%M"])#,minutes=["%m/%d %H:%M"])
    # set the grid interval filling
p.xgrid.band_fill_color = "grey"
p.xgrid.band_fill_alpha = 0.05
show(p)

In [98]:
os.getcwd()

'C:\\Users\\z0043H7B\\Tien\\Data\\dubuque\\Dubuque_080819\\Wacker'